In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from os.path import abspath, curdir, join, exists
import gffutils

In [5]:
data_dir = abspath(join(curdir, "..", "data"))

GCF_034236695.1.gff from the [Oner_Uvic_2.0](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_034236695.1/) assembly of Oncorhynchus nerka

In [ ]:
recreate = True
if not exists(join(data_dir, "GCF_034236695.1.db")) or recreate:
    db = gffutils.create_db(
        join(data_dir, "GCF_034236695.1.gff"),
        join(data_dir, "GCF_034236695.1.db"),
        merge_strategy="merge",
        force=True,
    )

: 